# HNSW Index Tuning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/configs/hnsw_tuning.ipynb)

Configure HNSW parameters for optimal recall/speed trade-offs. Covers m, ef_construct, and on-disk settings.

In [ ]:
!pip install -q qdrant-client

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import (
    VectorParams,
    Distance,
    HnswConfigDiff,
    OptimizersConfigDiff,
)

In [ ]:
client = QdrantClient(":memory:")

In [ ]:
# High-recall configuration (slower indexing, better search)
client.create_collection(
    collection_name="high_recall",
    vectors_config=VectorParams(
        size=768,
        distance=Distance.COSINE,
    ),
    hnsw_config=HnswConfigDiff(
        m=32,              # more connections = better recall, more memory
        ef_construct=256,  # higher = better index quality, slower build
    ),
    optimizers_config=OptimizersConfigDiff(
        indexing_threshold=50000,  # delay indexing until 50k points
    ),
)
print("Created high_recall collection")

info = client.get_collection("high_recall")
print(f"  HNSW m={info.config.hnsw_config.m}, ef_construct={info.config.hnsw_config.ef_construct}")

In [ ]:
# Fast configuration (lower recall, faster search)
client.create_collection(
    collection_name="fast_search",
    vectors_config=VectorParams(
        size=384,
        distance=Distance.COSINE,
    ),
    hnsw_config=HnswConfigDiff(
        m=16,
        ef_construct=100,
    ),
)
print("Created fast_search collection")

info = client.get_collection("fast_search")
print(f"  HNSW m={info.config.hnsw_config.m}, ef_construct={info.config.hnsw_config.ef_construct}")

In [ ]:
# Update HNSW config on existing collection
client.update_collection(
    collection_name="fast_search",
    hnsw_config=HnswConfigDiff(
        ef_construct=200,  # rebuild index with higher quality
    ),
)

info = client.get_collection("fast_search")
print(f"Updated fast_search: ef_construct={info.config.hnsw_config.ef_construct}")